In [6]:
import chardet

with open('Chat.csv', 'rb') as f:
    result = chardet.detect(f.read())

print(result['encoding'])


Windows-1252


## Data Preprocessing  

In [25]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sumy.parsers.plaintext import PlaintextParser
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.nlp.tokenizers import Tokenizer
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation



In [26]:
# import pandas as pd
df = pd.read_csv('Chat.csv', encoding='Windows-1252')


In [27]:
df.head()

,Question,Answer,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 86,Unnamed: 87,Unnamed: 88,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94,Answers(411)
0,What are the requirements for voting by absent...,Voters unable to vote in person on Election Da...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Absentee voting is available if you meet any o...
1,What is the voter registration deadline?,"Primary Election Date is June 7, 2022 (Registe...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,n person registration at the county clerk's of...
2,Where can I cast my vote?,"After registering to vote, your Voter Registra...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,What are the registration qualifications to vote?,Every U.S. citizen who possesses the following...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,How to register by mail to vote?,1. Complete a Mail-In Voter Registration Appli...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
# Select columns that have a name
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

# Remove columns that have only NaN values
df = df.dropna(axis=1, how='all')

In [29]:
df.head()

,Question,Answer,Answers(411)
0,What are the requirements for voting by absent...,Voters unable to vote in person on Election Da...,Absentee voting is available if you meet any o...
1,What is the voter registration deadline?,"Primary Election Date is June 7, 2022 (Registe...",n person registration at the county clerk's of...
2,Where can I cast my vote?,"After registering to vote, your Voter Registra...",NaN
3,What are the registration qualifications to vote?,Every U.S. citizen who possesses the following...,NaN
4,How to register by mail to vote?,1. Complete a Mail-In Voter Registration Appli...,NaN


In [30]:
def preprocess_text(text):
    # Loading spaCy model
    nlp = spacy.load('en_core_web_sm')

    # Check if text is a string
    if isinstance(text, str):
        # Converting text to lowercase
        text = text.lower()

        # Removing extra whitespaces
        text = ' '.join(text.split())

        # Removing punctuation
        text = re.sub(r'[^\w\s]', '', text)

        # Lemmatization
        doc = nlp(text)
        text = ' '.join([token.lemma_ for token in doc])

    return text


In [31]:
# Preprocessing text data
df['preprocessed_text'] = df['Answer'].apply(preprocess_text)


In [32]:
# Saving preprocessed data as a new CSV file
df.to_csv('preprocessed_file.csv', index=False)